In [ ]:
! pip install mysql-connector-python
import mysql.connector
import pandas as pd
import json

In [2]:
con = open('ref_con/db-prima', 'r').read().splitlines()
how = input('how to connect(local/public):')

mydb = mysql.connector.connect(
  host=con[0] if how == 'public' else con[1],
  user=con[2],
  password=con[3],
  database=con[4],
  autocommit=True
)

mycursor = mydb.cursor(dictionary=True)

In [58]:
def exec_q(qT):
  mycursor.execute(qT)
  myresult = mycursor.fetchall()
  jD = []
  for x in myresult:
    jD.append(x['json_result'])
  JReturn = '['+','.join(jD)+']'
  return JReturn

def exec_to_result(q):
  mycursor.execute(q)
  return mycursor.fetchall()

def get_erm_result(code,v='',limit=''):
    verQ = ' TRUE ' if v=='' else ' versi = "'+v+'" '
    limQ = '' if limit=='' else ' LIMIT '+limit 
    return exec_q("select json_result from primaprod.mr_result inner join primaprod.mr_form on \
id_form=mr_form.id where kode like '"+code+"' AND "+verQ+limQ)

def get_erm_result_from_RM(rm):
  #dapatkan barisan DR
  rawPKP = exec_to_result("select poli_kunjungan_pasien.attr_tambahan,poli_kunjungan_pasien.id _id from poli_kunjungan_pasien inner join pendaftaran\
    using (no_reg) where pendaftaran.no_rm = '"+rm+"' and poli_kunjungan_pasien.status_ok = 1\
      and attr_tambahan like '%\"emr\":%'")
  return get_attr_emr(rawPKP)

def get_erm_result_from_NOREG(nr):
  #dapatkan barisan attr
  rawPKP = exec_to_result("select attr_tambahan, poli_kunjungan_pasien.id _id from poli_kunjungan_pasien where no_reg = '"+nr+"' and status_ok = 1\
     and attr_tambahan like '%\"emr\":%'")
  return get_attr_emr(rawPKP)

def get_erm_result_from_DR(dr):
  rawPKP = exec_to_result("select attr_tambahan,poli_kunjungan_pasien.id _id from poli_kunjungan_pasien where id = '"+dr+"' and status_ok = 1\
    and attr_tambahan like '%\"emr\":%'")
  return get_attr_emr(rawPKP)

def parse_code_to_key(dic):
  rDic = {}
  for i in dic:
    jR = json.loads(i['json_result'])
    for k in jR:
      rDic[i['kode']+'.'+str(i['user_created'])+'.'+str(i['id'])+'.'+k] = []
      rDic[i['kode']+'.'+str(i['user_created'])+'.'+str(i['id'])+'.'+k].append(jR[k])
  return rDic

def get_attr_emr(attrs):
  EMRid, PKPid = [],[]
  re = {}
  for i in attrs:
    attr = json.loads(i['attr_tambahan'])
    PKPid.append(i['_id'])
    for j in attr['emr']:
      EMRid.append(j)
  
  mycursor.execute('select mr_result.id,json_result,kode,user_created from primaprod.mr_result inner join primaprod.mr_form on id_form=mr_form.id where mr_result.id in ("'+'","'.join([str(int) for int in EMRid])+'") limit 1')
  
  re = parse_code_to_key(mycursor.fetchall())
  
  # soap
  soap = get_soap(PKPid)
  if soap != 0:
    re = {**re,**soap}

  # resume2
  rsm2 = get_resume2(PKPid)
  if rsm2 != 0:
    re = {**re,**rsm2}

  # resume3
  rsm3 = get_resume3(PKPid)
  if rsm3 != 0:
    re = {**re,**rsm3}

  # notes
  notes = get_notes(PKPid)
  if notes != 0:
    re = {**re,**notes}

  return re

def get_soap(id):
  mycursor.execute('select id,"SOAP" kode, subjektif, objektif, assesment, planning, diagnosa_utama, diagnosa_sekunder, tindakan, user_input user_created from mr_soap_harian where detail_reg in ("'+'","'.join([str(int) for int in id])+'")')
  R = mycursor.fetchall()
  if R != None:
    return parse_other(R,['subjektif', 'objektif', 'assesment', 'planning', 'diagnosa_utama', 'diagnosa_sekunder', 'tindakan'])
  return 0

def get_resume2(id):
  mr = ['dokter_1','dokter_2','dokter_3','dokter_4','alasan_dirawat','diagnosa_utama','diagnosa_sekunder', 'penyebab_kematian', 'pemeriksaan_fisik', 'lab_penting', 'radiologi', 'penunjang_lain', 'tindakan', 'pengobatan_rawat', 'keadaan_pulang', 'keadaan_rujuk', 'keadaan_rujuk_lain', 'instruksi', 'kontrol_tanggal', 'kontrol_poli', 'diet', 'latihan', 'terapi_pulang', 'keadaan_kembali_igd', 'json_part']
  mycursor.execute('select id,"RM02" kode,'+",".join(mr)+' , user_input user_created from mr_resume_2 where detail_reg in ("'+'","'.join([str(int) for int in id])+'")')
  R = mycursor.fetchall()
  if R != None:
    return parse_other(R,mr)
  return 0

def get_resume3(id):
  mr = ['anamnesa', 'pemeriksaan_fisik', 'pemeriksaan_penunjang', 'diagnosa_utama', 'diagnosa_sekunder', 'tindakan', 'terapi', 'instruksi', 'keadaan_pulang', 'keadaan_rujuk', 'keadaan_rujuk_lain', 'json_part']
  mycursor.execute('select id,"RM03" kode,'+",".join(mr)+' , user_input user_created from mr_resume_3 where detail_reg in ("'+'","'.join([str(int) for int in id])+'")')
  R = mycursor.fetchall()
  if R != None:
    return parse_other(R,mr)
  return 0

def get_notes(id):
  mycursor.execute('select notes_json,detail_reg from primaprod.mr_notes where notes_json like "%notesmedis%" and detail_reg in ("'+'","'.join([str(int) for int in id])+'")')
  dic = mycursor.fetchall()
  if dic == None: return 0
  rDic = {}
  for i in dic:
    jR = json.loads(i['notes_json'])
    for k in jR['notesmedis']:
      rDic['N.'+k['Uid']+'.'+str(i['detail_reg'])+'.Tx'] = []
      rDic['N.'+k['Uid']+'.'+str(i['detail_reg'])+'.Tx'].append(k['Tx'])
  return rDic

def parse_other(dic,ar):
  rDic = {}
  for i in dic:
    for k in ar:
      rDic[i['kode']+'.'+str(i['user_created'])+'.'+str(i['id'])+'.'+k] = []
      rDic[i['kode']+'.'+str(i['user_created'])+'.'+str(i['id'])+'.'+k].append(i[k])
  return rDic


In [4]:
DFresult = pd.read_json(get_erm_result(input('KODE DOK RM:'),''))

In [ ]:
DFresult[(DFresult['detail_reg'] == 3316905)].tail()

In [ ]:
get_erm_result_from_DR('3368424')

In [16]:
dtdic = get_erm_result_from_DR('3296830')
df = pd.DataFrame(dtdic)